In [18]:
from textgenrnn import textgenrnn
import jamspell

from collections import Counter

In [2]:
corrector = jamspell.TSpellCorrector()
corrector.LoadLangModel('/home/mitya/PycharmProjects/text_gen/models/ru_small.bin')

True

In [3]:
textgen = textgenrnn('/home/mitya/PycharmProjects/text_gen/models/bashe_300.hdf5')

In [4]:
# large model, need more comppower
#model = KeyedVectors.load('/home/mitya/PycharmProjects/text_gen/models/187/model.model')

In [5]:
letters = "А,Б,В,Г,Д,Е,Ё,Ж,З,И,Й,К,Л,М,Н,О,П,Р,С,Т,У,Ф,Х,Ц,Ч,Ш,Щ,Ъ,Ы,Ь,Э,Ю,Я"
letters = letters.lower()
letters = letters.split(',')

In [70]:

def hokku_str(n):
    hokku = []
    for i in range(n):
        hokku.append(textgen.generate(return_as_list=True, temperature=1.0))
    
    while([''] in hokku) : 
        hokku.remove(['']) 
    
    hokku = list(filter(None, hokku))

    hokku_str = ''.join(map(str, hokku))
    hokku_str = hokku_str.replace('[','').replace(']', '').replace('    ', '')
    hokku_str = hokku_str.split("'")
    
    return hokku_str


def generate_poems(num_poems, num_str):
    '''generate list of poems'''
    poems = [None]
    for i in range(num_poems):
        poems[i] = hokku_str(num_str)
        poems.append(poems[i])
    return poems

    
def delete_empty(letters, x): 
    for letter in letters:
        if  letter in x: 
            return True
        else: 
            return False


def filter_empty(poems):
    '''filter empty lists'''
    for i in range(len(poems)):
        poems[i] = [element for element in poems[i] if delete_empty(letters, element) == True]
    return poems

        
def correct(poems):
    for i in range(len(poems)):
        for j in range(len(poems[i])):
            poems[i][j] = corrector.FixFragment(str(poems[i][j]))
    return poems


def generate_end():
    text = generate_poems(num_poems, num_str)
    result = filter_empty(text)
    result = correct(result)
    return result


In [11]:
num_str = 10
num_poems = 3

In [71]:
text = generate_end()

In [72]:
text

[['Откуда кукушки крик?', 'Как радует серде она.', 'Одной их, что транс'],
 ['Под тим славным летом',
  'Откуда кукушки крик?',
  'Первый лист упал…',
  'Сквозь чау густого бамбука'],
 ['адумалась луна.'],
 ['адумалась луна.']]

In [39]:
def list2string(s):
    str1 = " "    
    return (str1.join(s)) 


In [102]:
def count_words(text):
    b = 0
    for i in range(len(text)):
        a = len(list2string(text[i]).split())
        b += a
    return b


def count_words_cmplx(text):
    c = 0
    for i in range(len(text)):
        e = count_words(text[i])
        c += e
    return c

In [73]:
for i in range(len(text)):
    print(len(list2string(text[i]).split()))

11
14
2
2


In [74]:
count_words(text)

29

In [104]:
# generate poems up until some limit in words

def generate_novel(limit):
    text = []
    n_words = 0
    while True:
        #text[i] = generate_end()
        text.append(generate_end())
        n_words = count_words_cmplx(text)
    
        if n_words > limit:
            break
    return text


In [83]:
def generate_novel_test(n):
    text = []
    n_words = 0
    for i in range(n):
        #text[i] = generate_end()
        text.append(generate_end())
        
    return text


In [85]:
test = generate_novel_test(3)

In [86]:
len(test)

3

In [107]:
count_words(test[0]) + count_words(test[1]) + count_words(test[2])

121

In [103]:
count_words_cmplx(test)

121

In [108]:
generate_novel(100)

[[['бабочки полет',
   'соломенный плаик надеть…',
   'Пода коровы друга',
   'акруит, несет с собой',
   'Как будто слезы…'],
  ['Прощаясь с друзьями',
   'Какая грусть!',
   'В лугах привольных',
   'Как тел первый снег!'],
  ['очь напролет кругом!',
   'Плачут птиц. лаза у рыб',
   'расставаясь с другом',
   'долгий день напролт'],
  ['очь напролет кругом!',
   'Плачут птиц. лаза у рыб',
   'расставаясь с другом',
   'долгий день напролт']],
 [['сть особая прелесть'],
  ['В небе такая луна,',
   'а склонах Каменной горы',
   'Под снегом бамбук.',
   'От, потерявему сына',
   'апочку из цветов?'],
  ['был ли он славой когда-то?', 'О вини старого сада!'],
  ['был ли он славой когда-то?', 'О вини старого сада!']],
 [['апл по своему ниву.', 'Измученный, до ночлега…', 'ам – за бамбука'],
  ['Возвращается к старым корням…',
   ' коня на спине.',
   'Стебельки лука,',
   'Пой типу песни за чакой похлебки,'],
  ['пала в светло волну',
   'апочку из цветов?',
   'Слух, как градины стучат.',
